In [1]:
!pip install torch spacybert

In [2]:
!unzip spacycake.zip

Archive:  spacycake.zip
replace spacycake/about.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: spacycake/about.py      
  inflating: spacycake/__init__.py   


In [3]:
# Based on https://prakhartechviz.blogspot.com/2020/07/unsupervised-keyword-extraction-using-sentence-embeddings.html
# Reference: http://www.cs.bilkent.edu.tr/~canf/CS533/hwSpring14/eightMinPresentations/handoutMMR.pdf
from spacybert import BertInference
from spacy.tokens import Doc
from spacy.matcher import Matcher
from spacy.util import filter_spans
from spacycake import BertKeyphraseExtraction as bake
import spacy
import torch
from typing import Tuple

In [4]:
nlp = spacy.load('en')
doc = nlp("""Keywords/Keyphrase extraction is the task of extracting relevant and representative words that best describe the underlying document. Keywords extraction has many use-cases, some of which are using it as meta-data for indexing documents and later using in IR systems, it also plays as a crucial component when gleaning real-time insights.

Although, in literature, this problem has been tried in both supervised and unsupervised settings. This paper proposes EmbedRank - An embedding based unsupervised keyphrase extraction technique from a single document followed by an extra step of re-ranking the candidate phrases for selecting the final set. The proposed technique takes care of explicitly increasing the diversity yet preserving the coverage of the selected candidate keywords. Authors talk about the limitations of existing keyphrase extractor systems which is of generating morphologically and semantic redundant phrases, for example - "molecular equivalence numbers" and "molecular equivalence indices" mean the same thing and represent the same concept. Getting both of these keywords in output does not necessarily convey anything extra and might hamper user experience. Authors employ Maximal Marginal Relevance (MMR) ranking strategy as trade-off between diversity and relevance.

In short, they propose the technique of representing both document and candidates phrases in a common high-dimensional vector space using existing embeddings model and then later computing meaningful distances between them to ensure informativeness as well as the distance between candidates as the measure of diversity in the final selection.

The method consists of 4 main steps -
Selecting the first set of candidate phrases from the document based on certain Part-of-Speech patterns. (They keep only phrases that follows the pattern JJ*NN+ (Zero of more adjectives followed by one or multiple nouns)) (Ensures Noise Reduction)
Vectorization of the document and every candidate phrases extracted from Step.1 into a common high-dimensional vector space. 
Selecting top-k candidates by calculating the cosine similarity between phrases and the document (Ensures Informativeness)
Re-ranking the top-k selected candidates using MMR (Maximal Marginal Relevance) and then selecting top-n phrases. (Ensures Diversity)
Authors talk about experimenting with pre-trained Doc2Vec and Sent2Vec models as the choice for embedding documents and candidate phrases in a common embedding space of 700 dimensions. After experimenting with the embeddings they found both the methods to be providing comparable vectors no matter if the input document is a word, sentence, long document. They also mention about Sent2Vec being faster compared to Doc2Vec during inference.

Authors compared their results with existing keyword algorithms on various datasets such as Inspec, DUC 2001, and NUS and found their results to be surpassing current SOTA on Inspec and DUC 2001.""")
doc

Keywords/Keyphrase extraction is the task of extracting relevant and representative words that best describe the underlying document. Keywords extraction has many use-cases, some of which are using it as meta-data for indexing documents and later using in IR systems, it also plays as a crucial component when gleaning real-time insights.

Although, in literature, this problem has been tried in both supervised and unsupervised settings. This paper proposes EmbedRank - An embedding based unsupervised keyphrase extraction technique from a single document followed by an extra step of re-ranking the candidate phrases for selecting the final set. The proposed technique takes care of explicitly increasing the diversity yet preserving the coverage of the selected candidate keywords. Authors talk about the limitations of existing keyphrase extractor systems which is of generating morphologically and semantic redundant phrases, for example - "molecular equivalence numbers" and "molecular equivale

In [5]:
cake = bake(nlp, from_pretrained='DeepPavlov/bert-base-cased-conversational')
nlp.add_pipe(cake)
print(nlp.pipe_names)

INFO:transformers.tokenization_utils_base:Model name 'DeepPavlov/bert-base-cased-conversational' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, TurkuNLP/bert-base-finnish-cased-v1, TurkuNLP/bert-base-finnish-uncased-v1, wietsedv/bert-base-dutch-cased). Assuming 'DeepPavlov/bert-base-cased-conversational' is a path, a model identifier, or url to a directory containing tokenizer files.
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/DeepPavlov/bert-base-cased-conversational/vocab.txt from 

['tagger', 'parser', 'ner', 'bert_keyphrase_extraction']


In [71]:
phrases = cake._get_candidate_phrases(doc)
print(phrases)

[extraction, task, representative words, document, Keywords extraction, many use, cases, meta, data, indexing documents, systems, crucial component, time insights, literature, problem, unsupervised settings, paper, embedding, unsupervised keyphrase extraction technique, single document, extra step, re-, candidate phrases, final set, technique, care, diversity, coverage, candidate keywords, Authors, limitations, extractor systems, semantic redundant phrases, example, molecular equivalence numbers, molecular equivalence indices, same thing, same concept, keywords, output, user experience, Authors, strategy, trade, off, diversity, relevance, technique, document, candidates phrases, dimensional vector space, embeddings model, meaningful distances, informativeness, distance, candidates, measure, diversity, final selection, method, main steps, first set, candidate phrases, document, certain Part, Speech patterns, phrases, pattern JJ*NN+, more adjectives, multiple nouns, Vectorization, docume

In [72]:
# Get document embedding
doc_embedding = getattr(doc._, cake.attr_names[0])

In [8]:
from __future__ import print_function
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)

In [73]:
import sys
import time
phrases_embeddings = []
for i, p in enumerate(phrases):
    start = time.time()
    tensor = getattr(p._, cake.attr_names[0])
    phrases_embeddings.append(tensor.tolist())
    print(i, p, '|| time to run:', time.time() - start, '|| size:', total_size(tensor), "|| size of phrases_embeddings list:", total_size(phrases_embeddings))

0 extraction || time to run: 2.1029794216156006 || size: 72 || size of phrases_embeddings list: 24736
1 task || time to run: 2.043354034423828 || size: 72 || size of phrases_embeddings list: 49376
2 representative words || time to run: 2.0594043731689453 || size: 72 || size of phrases_embeddings list: 74016
3 document || time to run: 2.053642749786377 || size: 72 || size of phrases_embeddings list: 98656
4 Keywords extraction || time to run: 2.095147132873535 || size: 72 || size of phrases_embeddings list: 123328
5 many use || time to run: 2.074693202972412 || size: 72 || size of phrases_embeddings list: 147968
6 cases || time to run: 2.0458600521087646 || size: 72 || size of phrases_embeddings list: 172608
7 meta || time to run: 2.0508527755737305 || size: 72 || size of phrases_embeddings list: 197248
8 data || time to run: 2.0721216201782227 || size: 72 || size of phrases_embeddings list: 221952
9 indexing documents || time to run: 2.0377750396728516 || size: 72 || size of phrases_em

In [75]:
phrases_embeddings = torch.Tensor(phrases_embeddings)
start = time.time()
doc_phrase_similarity = torch.matmul(doc_embedding, phrases_embeddings.transpose(0, 1))
print(time.time() - start)

0.0011990070343017578


In [76]:
# Rank phrases based on similarity ascending
# and limit to top-k phrases
indices = doc_phrase_similarity.argsort(descending=True)
phrases_embeddings = phrases_embeddings[indices]
phrases = [phrases[i] for i in indices.tolist()]
R = list(range(len(phrases)))
S = []

In [77]:
phrases_embeddings.shape

torch.Size([106, 768])

In [78]:
# Re-rank based on Maximum Marginal Relevance score
start = time.time()
while len(R) > 0:
    first_part = torch.matmul(doc_embedding, phrases_embeddings[R].transpose(0, 1))
    second_part = torch.matmul(
        phrases_embeddings[R],
        phrases_embeddings[S].transpose(0, 1)).max(dim=1).values if S else torch.Tensor([0.] * first_part.shape[0])
    scores = (cake.mmr_lambda * first_part) - ((1 - cake.mmr_lambda) * second_part)
    phrase_to_add = R[scores.argmax()]
    R.remove(phrase_to_add)
    S.append(phrase_to_add)
print(time.time() - start)

0.904360294342041


In [79]:
top_k = 5
print([phrases[i] for i in S[:cake.top_k]])

[molecular equivalence indices, unsupervised keyphrase extraction technique, k candidates, single document, dimensional vector space]
